<a href="https://colab.research.google.com/github/SushmithaUW/FNC-1-Stance-Detection/blob/master/Predictions_on_Competetion_Unlabelled_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bidirectional LSTM with W2V and GloVe**


In [0]:
!unzip fnc-1.zip
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip


In [0]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config)

In [0]:
# Specify the folder locations
W2V_DIR = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
DATA_DIR = 'fnc-1/'
Glove = 'glove.6B.200d.txt'


In [0]:
import gensim
import keras
import numpy as np
import pandas as pd
import pickle
from keras.utils import np_utils


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors



import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.preprocessing import LabelEncoder



np.random.seed(1003)

In [0]:
MAX_SENT_LEN = 170
MAX_VOCAB_SIZE = 400000
LSTM_DIM = 128
EMBEDDING_DIM = 200
#EMBEDDING_DIM = 300
BATCH_SIZE = 200
N_EPOCHS = 10

In [0]:
competetion_bodies = pd.read_csv(DATA_DIR+'competition_test_bodies.csv')
competetion_stances = pd.read_csv(DATA_DIR+'competition_test_stances.csv')

competetion_unlabeled = pd.read_csv(DATA_DIR+'competition_test_stances_unlabeled.csv')


In [0]:
comp = competetion_stances.join(competetion_bodies.set_index('Body ID'), on='Body ID')

In [0]:
comp.replace('unrelated',1,True)
comp.replace('agree',2,True)
comp.replace('disagree',3,True)
comp.replace('discuss',4,True)

In [0]:
word_seq_headline_comp = [text_to_word_sequence(sent) for sent in comp['Headline']]
word_seq_bodies_comp = [text_to_word_sequence(sent) for sent in comp['articleBody']]


In [0]:
word_seq = []
for i in range(len(word_seq_bodies_comp)):
  word_seq.append(word_seq_bodies_comp[i])
  
for i in range(len(word_seq_bodies_comp)):
  word_seq.append(word_seq_bodies_comp[i])

In [0]:
for i in range (len(word_seq_headline_comp)):
  word_seq_headline_comp[i].extend(word_seq_bodies_comp[i])



In [0]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq])


In [0]:
X_comp = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_headline_comp])
X_comp = pad_sequences(X_comp, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
y_comp = comp['Stance']
y_comp = y_comp.values


In [0]:
encoder_comp = LabelEncoder()
encoder_comp.fit(y_comp)
encoded_comp = encoder_comp.transform(y_comp)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_comp = np_utils.to_categorical(encoded_comp)

In [81]:
from keras import models
model = models.load_model('BiLSTM.h5')
y_pred = model.predict(X_comp)
score,acc = model.evaluate(X_comp, dummy_y_comp)
print (score)


W0723 18:26:37.060389 140507819177856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 18:26:37.069651 140507819177856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 18:26:37.085984 140507819177856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 18:26:38.161896 140507819177856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0723 18:26:38.170127 

25413/25413 [==============================] - 82s 3ms/step
0.8605693960698311


In [0]:
outputs = [np.argmax(p, axis = -1) for p in y_pred]
for i in range(len(outputs)):
    if outputs[i] == 0: outputs[i] = "unrelated"
    if outputs[i] == 1: outputs[i] = "disagree"
    if outputs[i] == 2: outputs[i] = "agree"
    if outputs[i] == 3: outputs[i] = "discuss"

In [0]:
Predicted = {}
Predicted = pd.DataFrame({'Stance': outputs})
result = pd.concat([competetion_unlabeled, Predicted], axis=1, sort=False)
result.to_csv('./fnc-1/competetion_labelled.csv', index=False, encoding='utf-8')